### use vgg16 model to train dog vs cat
This model REALLY needs to use GPU


In [1]:
import numpy as np
from tensorflow import keras
import glob
import cv2

In [2]:
# read one image to veify size
test_images = glob.glob('test/*/*.jpg')
train_images = glob.glob('train/*/*.jpg')

single_image = test_images[0]
img = cv2.imread(single_image)
h,w,c = img.shape
input_shape=(h,w,c)

print('input shape:', input_shape)

input shape: (224, 224, 3)


In [3]:
# define the model
def setup_vgg16_model(input_shape):
    
    model = keras.applications.vgg16.VGG16(include_top='False', input_shape=input_shape)
    # make all layers are not trainable
    
    for layer in model.layers:
        layer.trainable = False
        
    #model.summary()
    
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    
    # we want to connext out own dense after fc2
    x = layer_dict['fc2'].output
    
    # new output is adding flatten , dense(128) , and dense(1)
    
    x = keras.layers.Dense(128,activation='relu')(x)
    x = keras.layers.Dense(1,activation='sigmoid')(x)
    
    mymodel = keras.models.Model(inputs=model.input , outputs=x)
    opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
    mymodel.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return mymodel


In [4]:
model = setup_vgg16_model(input_shape)

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [ ]:

batch_size=64

datagen = keras.preprocessing.image.ImageDataGenerator(rescale = (1.0/255.0))
                                                       
train_iter = datagen.flow_from_directory('train/',class_mode='binary', target_size=(h,h), batch_size=batch_size)
test_iter = datagen.flow_from_directory('test/',class_mode='binary', target_size=(h,h), batch_size=batch_size)

print('length of iterations for train,test:', len(train_iter), len(test_iter))
                                                     

Found 18711 images belonging to 2 classes.
Found 6235 images belonging to 2 classes.
length of iterations for train,test: 293 98


In [ ]:
#train the model with train data
fit_history = model.fit_generator(train_iter, 
                                  steps_per_epoch=len(train_iter), 
                                  #workers=2,
                                  epochs=10,
                                  validation_data=test_iter,
                                  validation_steps=len(test_iter)
                    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
  1/293 [..............................] - ETA: 1:00:35 - loss: 0.7155 - acc: 0.6094